In [113]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
import matplotlib.pyplot as plt

In [114]:
# Load the California Housing dataset to get the feature names
california = fetch_california_housing()

In [115]:
# Load the pre-trained model
pretrained_model = load_model('california_housing_model.keras')

In [116]:
# Freeze the layers of the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

In [117]:
# Load additional dataset
# For demonstration, we create synthetic additional data. Replace this with the actual additional data loading process.
additional_data_size = 100
additional_data = pd.DataFrame(np.random.rand(additional_data_size, pretrained_model.input_shape[1]), columns=california.feature_names)
additional_data['PRICE'] = np.random.rand(additional_data_size) * 5  # Random prices for example


In [118]:
# Combine the original and additional datasets
original_data = pd.DataFrame(california.data, columns=california.feature_names)
original_data['PRICE'] = california.target
combined_data = pd.concat([original_data, additional_data], ignore_index=True)

In [119]:
# Split the combined data into features and target
X_combined = combined_data.drop('PRICE', axis=1)
y_combined = combined_data['PRICE']

In [120]:
# Normalize the combined data
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

In [121]:
# Split the combined data into training and test sets
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)


In [122]:
# Add new layers on top of the frozen base model
new_model = Sequential([
    pretrained_model,  # Add the base model
    Dense(32, activation='relu'),  # New dense layer
    Dense(1)  # Output layer
])

In [123]:
# Compile the new model
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')


In [124]:
# Train the new model on the combined dataset (fine-tuning)
history = new_model.fit(X_train_combined, y_train_combined, epochs=100, validation_split=0.2)

Epoch 1/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 1s 639us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/100
415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 11/100
415/415 ━━━━━━━━

In [125]:

# Evaluate the fine-tuned model
fine_tuned_pred = new_model.predict(X_test_combined)
fine_tuned_mae = mean_absolute_error(y_test_combined, fine_tuned_pred)
fine_tuned_rmse = mean_squared_error(y_test_combined, fine_tuned_pred, squared=False)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step


C:\Users\Comp2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [126]:
# Print the performance of the fine-tuned model
print(f"Fine-Tuned Neural Network MAE: {fine_tuned_mae}, RMSE: {fine_tuned_rmse}")

Fine-Tuned Neural Network MAE: 0.7016937154817562, RMSE: 0.9161775087645699
